In [41]:
import pandas as pd
import functools
import operator
import numpy as np

In [146]:
import json
import logging
import time
import re
from pathlib import Path
from urllib.parse import urljoin

import cloudscraper
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from tenacity import retry, wait_fixed, stop_after_attempt

d = []

COLLECTIONS_URL = [f"https://www.goat.com/sneakers?release_date_year={year}" for year in range(1985, 2026)]

ua      = UserAgent()
scraper = cloudscraper.create_scraper(
    browser={"browser": "firefox", "platform": "windows", "desktop": True},
    delay=10,
)
HEADERS = {"Accept-Language": "en-US,en;q=0.9"}

@retry(wait=wait_fixed(2), stop=stop_after_attempt(3))
def fetch(url: str) -> str:
    resp = scraper.get(url, headers={"User-Agent": ua.random, **HEADERS}, timeout=30)
    resp.raise_for_status()
    return resp.text


def get_listing_data() -> list[dict]:
    items = []
    for url in COLLECTIONS_URL:
        html = fetch(url)
        soup = BeautifulSoup(html, "lxml")
        for cell in soup.select('div[data-qa="grid_cell_product"]'):
            a = cell.select_one('a[href^="/sneakers/"]')
            if not a:
                continue
            url = urljoin(url, a["href"].split("?")[0])
            price_tag = cell.select_one('[data-qa="grid_cell_product_price"] .LocalizedCurrency__Amount-sc-yoa0om-0')
            price = None
            if price_tag:
                text = price_tag.get_text(strip=True)
                price = float(text.replace("$", "").replace(",", ""))
            items.append({"url": url, "price": price})
    return items
NEXT_DATA_RE = re.compile(r'<script[^>]+id="__NEXT_DATA__"[^>]*>(.+?)</script>', re.S)

def load_next_data(html: str) -> dict:
    m = NEXT_DATA_RE.search(html)
    if not m:
        raise ValueError("⛔ __NEXT_DATA__ not found")
    return json.loads(m.group(1))

def deep_find(obj, want_keys: set[str]):
    """Рекурсивно ищем dict, в котором есть все want_keys."""
    if isinstance(obj, dict):
        if want_keys.issubset(obj.keys()):
            return obj
        for v in obj.values():
            hit = deep_find(v, want_keys)
            if hit:
                return hit
    elif isinstance(obj, list):
        for item in obj:
            hit = deep_find(item, want_keys)
            if hit:
                return hit
    return None


def parse_product_page(url: str) -> dict:
    """
    С товарной страницы извлекаем поля:
    Release Date, Designer, Model, Colorway, Main Color,
    Upper Material, Technology.
    """
    html = fetch(url)
    data = load_next_data(html)

    product = deep_find(data, {"brandName", "name"})
    if not product:
        raise ValueError("Product dictionary not found")
    d.append({
        "url":            url,
        "release_date":   product.get("releaseDate", "")[:10],
        "brand_name":       product.get("brandName"),
        "model":          product.get("name"),
        "gender":         product.get("gender"),
        "len_colorway":   len(functools.reduce(operator.iconcat, (a.split('/') for a in (product.get("details") or product.get("colorway")).split()), [])) if (product.get("details") or product.get("colorway")) is not None else 0,
        "main_color":     product.get("color"),
        "upper_material": product.get("upperMaterial"),
        "technology":     product.get("midsole") or product.get("technology"),
        "len_size": len(product.get("sizeRange")), 
        "unique": (np.any([key in product.get('storyHtml').lower() for key in ['unique', 'limited', 'collab']])).astype(int) if product.get('storyHtml') is not None else 0
    })
    return {
        "url":            url,
        "release_date":   product.get("releaseDate", "")[:10],
        "designer":     product.get("brandName"),
        "model":          product.get("name"),
        "gender":         product.get("gender"),
        "len_colorway":   product.get("details") or product.get("colorway"),
        "len_colorway":   product.get("details") or product.get("colorway"),
        "main_color":     product.get("color"),
        "upper_material": product.get("upperMaterial"),
        "technology":     product.get("midsole") or product.get("technology")
    }


def main(limit = 950, pause: float = 0.005):
    # 1) получаем список товаров + цен
    listing = get_listing_data()
    print(print(len(listing)), "items")
    if limit:
        listing = listing[:limit]

    results = []
    for i, item in enumerate(listing, 1):
        url, price = item["url"], item["price"]
        try:
            info = parse_product_page(url)
            d[-1]["price"] = price 
            print(len(d))
            # добавляем цену из списка
            info["price"] = price
            results.append(info)
            logging.info(
                "[%03d] %s – %s  price=%s",
                i, info["designer"], info["model"], price
            )
        except Exception as e:
            logging.warning("[%03d] ⚠️ %s : %s", i, url, e)
        time.sleep(pause)

    out = Path("goat_just_dropped.jsonl")
    out.write_text(
        "\n".join(json.dumps(x, ensure_ascii=False) for x in results),
        "utf-8"
    )
    print(f"\n✔️ Saved {len(results)} items → {out.resolve()}")

if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO, format="%(message)s")
    main()  # снимите limit=None, чтобы пройтись по всей коллекции

913
None items
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
2

In [176]:
data = pd.DataFrame(d)

In [183]:
data['gender'] = data['gender'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

In [184]:
data.drop_duplicates(subset=['url'], inplace=True)

In [185]:
data['days_since_release'] = (pd.to_datetime('today') - pd.to_datetime(data['release_date'])).dt.days

In [189]:
data.to_csv('goat_parsed.csv', index=False)

In [226]:
import json
import logging
import time
import re
from pathlib import Path
from urllib.parse import urljoin

import cloudscraper
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from tenacity import retry, wait_fixed, stop_after_attempt

d2 = []

COLLECTIONS_URL = [f"https://www.goat.com/search?query=wmns&is_in_stock=True&web_groups=sneakers&release_date_year={year}" for year in range(2003, 2026)]

ua      = UserAgent()
scraper = cloudscraper.create_scraper(
    browser={"browser": "firefox", "platform": "windows", "desktop": True},
    delay=10,
)
HEADERS = {"Accept-Language": "en-US,en;q=0.9"}

@retry(wait=wait_fixed(2), stop=stop_after_attempt(3))
def fetch(url: str) -> str:
    resp = scraper.get(url, headers={"User-Agent": ua.random, **HEADERS}, timeout=30)
    resp.raise_for_status()
    return resp.text


def get_listing_data() -> list[dict]:
    items = []
    for url in COLLECTIONS_URL:
        html = fetch(url)
        soup = BeautifulSoup(html, "lxml")
        for cell in soup.select('div[data-qa="grid_cell_product"]'):
            a = cell.select_one('a[href^="/sneakers/"]')
            if not a:
                continue
            url = urljoin(url, a["href"].split("?")[0])
            price_tag = cell.select_one('[data-qa="grid_cell_product_price"] .LocalizedCurrency__Amount-sc-yoa0om-0')
            price = None
            if price_tag:
                text = price_tag.get_text(strip=True)
                price = float(text.replace("$", "").replace(",", ""))
            items.append({"url": url, "price": price})
    return items
NEXT_DATA_RE = re.compile(r'<script[^>]+id="__NEXT_DATA__"[^>]*>(.+?)</script>', re.S)

def load_next_data(html: str) -> dict:
    m = NEXT_DATA_RE.search(html)
    if not m:
        raise ValueError("⛔ __NEXT_DATA__ not found")
    return json.loads(m.group(1))

def deep_find(obj, want_keys: set[str]):
    """Рекурсивно ищем dict, в котором есть все want_keys."""
    if isinstance(obj, dict):
        if want_keys.issubset(obj.keys()):
            return obj
        for v in obj.values():
            hit = deep_find(v, want_keys)
            if hit:
                return hit
    elif isinstance(obj, list):
        for item in obj:
            hit = deep_find(item, want_keys)
            if hit:
                return hit
    return None


def parse_product_page(url: str) -> dict:
    """
    С товарной страницы извлекаем поля:
    Release Date, Designer, Model, Colorway, Main Color,
    Upper Material, Technology.
    """
    html = fetch(url)
    data = load_next_data(html)

    product = deep_find(data, {"brandName", "name"})
    if not product:
        raise ValueError("Product dictionary not found")
    d2.append({
        "url":            url,
        "release_date":   product.get("releaseDate", "")[:10],
        "brand_name":       product.get("brandName"),
        "model":          product.get("name"),
        "gender":         product.get("gender"),
        "len_colorway":   len(functools.reduce(operator.iconcat, (a.split('/') for a in (product.get("details") or product.get("colorway")).split()), [])) if (product.get("details") or product.get("colorway")) is not None else 0,
        "main_color":     product.get("color"),
        "upper_material": product.get("upperMaterial"),
        "technology":     product.get("midsole") or product.get("technology"),
        "len_size": len(product.get("sizeRange")), 
        "unique": (np.any([key in product.get('storyHtml').lower() for key in ['unique', 'limited', 'collab']])).astype(int) if product.get('storyHtml') is not None else 0
    })
    return {
        "url":            url,
        "release_date":   product.get("releaseDate", "")[:10],
        "designer":     product.get("brandName"),
        "model":          product.get("name"),
        "gender":         product.get("gender"),
        "len_colorway":   product.get("details") or product.get("colorway"),
        "len_colorway":   product.get("details") or product.get("colorway"),
        "main_color":     product.get("color"),
        "upper_material": product.get("upperMaterial"),
        "technology":     product.get("midsole") or product.get("technology")
    }


def main(limit = 950, pause: float = 0.005):
    # 1) получаем список товаров + цен
    listing = get_listing_data()
    print(print(len(listing)), "items")
    if limit:
        listing = listing[:limit]

    results = []
    for i, item in enumerate(listing, 1):
        url, price = item["url"], item["price"]
        try:
            info = parse_product_page(url)
            d2[-1]["price"] = price 
            print(len(d2))
            # добавляем цену из списка
            info["price"] = price
            results.append(info)
            logging.info(
                "[%03d] %s – %s  price=%s",
                i, info["designer"], info["model"], price
            )
        except Exception as e:
            logging.warning("[%03d] ⚠️ %s : %s", i, url, e)
        time.sleep(pause)

    out = Path("goat_just_dropped_women.jsonl")
    out.write_text(
        "\n".join(json.dumps(x, ensure_ascii=False) for x in results),
        "utf-8"
    )
    print(f"\n✔️ Saved {len(results)} items → {out.resolve()}")

if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO, format="%(message)s")
    main()  # снимите limit=None, чтобы пройтись по всей коллекции

373
None items
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
2

In [259]:
data2 = pd.read_csv('goat_parsed_women.csv')
data2['gender'] = data2['gender'].apply(lambda x: 'women' if 'women' in x else 'men')
data2.drop_duplicates(subset=['url'], inplace=True)
data2['days_since_release'] = (pd.to_datetime('today') - pd.to_datetime(data2['release_date'])).dt.days
data2 = data2[data2['gender'] == 'women']
data2.to_csv('goat_parsed_women.csv', index=False)